In [1]:
import tensorflow as tf
import os
import numpy as np
import random
SEED = 0
#------------------------------------------------------------------------------------
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
#------------------------------------------------------------------------------------
def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)
#-----------------------------------------------------------------------------------

In [2]:
%matplotlib notebook
import pandas as pd
from data_extraction import *
from resp_signal_extraction import *
from rr_extration import *
from sklearn.preprocessing import MinMaxScaler
import re
import pickle as pkl
from tf_model import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
import matplotlib.pyplot as plt
from filters import *
import tqdm
import plotly as py
import plotly.figure_factory as ff
import ipywidgets as widgets
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
py.offline.init_notebook_mode(connected = True)
from plotly import tools
import plotly.express as px

In [3]:
input_conf = 'conff'

In [4]:
with open('output','rb') as f:
    output_data = pkl.load(f)

with open('input','rb') as f:
    input_data = pkl.load(f)

with open('raw_signal.pkl','rb') as f:
    raw_data = pkl.load(f)

input_data = np.transpose(input_data, (0,2,1))
raw_data = np.transpose(raw_data, (0,2,1))
annotation = pd.read_pickle('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/annotation.pkl')
reference_rr = (annotation['Reference_RR'].values).reshape(-1,1)

input_data = np.around(input_data , decimals = 4)
raw_data = np.around(raw_data , decimals = 4)
output_data = np.around(output_data , decimals = 4)
reference_rr = np.around(reference_rr , decimals = 4)

tensor_input = tf.convert_to_tensor(input_data, dtype = 'float32')
tensor_output = tf.convert_to_tensor(output_data, dtype = 'float32')
tensor_ref_rr = tf.convert_to_tensor(reference_rr, dtype = 'float32')
tensor_raw_data = tf.convert_to_tensor(raw_data, dtype = 'float32')
training_ids = annotation['patient_id'] < 13

x_train_data = tensor_input[tf.convert_to_tensor(training_ids.values)]
x_test_data = tensor_input[tf.convert_to_tensor(~(training_ids.values))]
x_train_ref_rr = tensor_ref_rr[tf.convert_to_tensor(training_ids.values)]
x_test_ref_rr = tensor_ref_rr[tf.convert_to_tensor(~(training_ids.values))]
x_train_raw_sig = tensor_raw_data[tf.convert_to_tensor(training_ids.values)]
x_test_raw_sig = tensor_raw_data[tf.convert_to_tensor(~(training_ids.values))]

y_train_data = tensor_output[tf.convert_to_tensor(training_ids.values)]
y_test_data = tensor_output[tf.convert_to_tensor(~(training_ids.values))]


2021-09-10 17:30:43.135355: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-10 17:30:43.140526: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-10 17:30:43.140854: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-10 17:30:43.141782: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
if input_conf == 'confc':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , y_train_data))
        train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , y_test_data))
        test_dataset = test_dataset.batch(128)

if input_conf == 'confb':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , x_train_ref_rr))
        train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , x_test_ref_rr))
        test_dataset = test_dataset.batch(128)

if input_conf == 'confd':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , y_train_data, x_train_ref_rr))
        train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , y_test_data, x_test_ref_rr))
        test_dataset = test_dataset.batch(128)
if input_conf == 'confa':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw_sig , x_train_ref_rr))
        train_dataset = train_dataset.shuffle(len(x_train_raw_sig)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw_sig , x_test_ref_rr))
        test_dataset = test_dataset.batch(128)

if input_conf == 'confe':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw_sig , y_train_data))
        train_dataset = train_dataset.shuffle(len(x_train_raw_sig)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw_sig , y_test_data))
        test_dataset = test_dataset.batch(128)
if input_conf == 'conff':
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw_sig , y_train_data, x_train_ref_rr))
        train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw_sig , y_test_data, x_test_ref_rr))
        test_dataset = test_dataset.batch(128)
    

In [6]:
if input_conf == 'confc':
    model_input_shape = (128,3)
    model  = BRUnet(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confc/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test,y_batch_test) in enumerate(test_dataset):
        output = model(x_batch_test)
        test_loss = loss_fn(y_batch_test , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
        
#-----------------------------------------------------------------------------------------------------------------
if input_conf == 'confd':
    model_input_shape = (128,3)
    model  = BRUnet_Multi_resp(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confd/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    final_output_rr = tf.convert_to_tensor([])
    for step , (x_batch_test,y_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
        test_output,test_out_rr = model(x_batch_test)
        test_loss_resp = loss_fn(y_batch_test  , test_output)
        test_loss_rr = loss_fn(x_batch_test_ref_rr , test_out_rr)
        test_loss = test_loss_resp + test_loss_rr
        if step == 0:
            final_output = test_output
            final_output_rr = test_out_rr
        else:
            final_output = tf.concat([final_output , test_output] , axis = 0)
            final_output_rr = tf.concat([final_output_rr , test_out_rr] , axis = 0)
        test_loss_list.append(test_loss)
            
#------------------------------------------------------------------------------------------------------------------
if input_conf == 'confb':
    model_input_shape = (128,3)
    model  = BRUnet_Encoder(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confb/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
        output = model(x_batch_test)
        test_loss = loss_fn(x_batch_test_ref_rr , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#--------------------------------------------------------------------------------------------------------------------
if input_conf == 'confe':
    model_input_shape = (2048,3)
    model  = BRUnet_raw(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confe/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test_raw,y_batch_test) in enumerate(test_dataset):
        output = model(x_batch_test_raw)
        test_loss = loss_fn(y_batch_test , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
        
#-------------------------------------------------------------------------------------------------------------------
if input_conf == 'confa':
    model_input_shape = (2048,3)
    model  = BRUnet_raw_encoder(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confa/best_model_100.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test_raw, x_batch_test_ref_rr) in enumerate(test_dataset):
        output = model(x_batch_test_raw)
        test_loss = loss_fn(x_batch_test_ref_rr , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#-------------------------------------------------------------------------------------------------------------------
if input_conf == 'conff':
    model_input_shape = (2048,3)
    model  = BRUnet_raw_multi(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/conff/best_model_100.h5')        
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    final_output_rr = tf.convert_to_tensor([])
    for step , (x_batch_test_raw , y_batch_test , x_batch_test_ref_rr) in enumerate(test_dataset):
        test_output,test_out_rr = model(x_batch_test_raw)
        test_loss_resp = loss_fn(y_batch_test  , test_output)
        test_loss_rr = loss_fn(x_batch_test_ref_rr , test_out_rr)
        test_loss = test_loss_resp + test_loss_rr
        if step == 0:
            final_output = test_output
            final_output_rr = test_out_rr
        else:
            final_output = tf.concat([final_output , test_output] , axis = 0)
            final_output_rr = tf.concat([final_output_rr , test_out_rr] , axis = 0)
        test_loss_list.append(test_loss)
    

2021-09-10 17:30:44.552234: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8202


# MONTE CARLO WITH SAMPLING

In [7]:
if input_conf == 'confc':
    final_output = np.array([])
    model_input_shape = (128,3)
    model  = BRUnet(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confc/best_model_100.h5')
    for i in tqdm.tqdm(range(10)):
        test_loss_list = []
        output_data = tf.convert_to_tensor([])
        for step,(x_batch_test , y_batch_test) in enumerate(test_dataset):
            output = model(x_batch_test)
            test_loss = loss_fn(y_batch_test,output)
            if step == 0:
                output_data = output
            else:
                output_data = tf.concat([output_data , output] , axis = 0)
            test_loss_list.append(test_loss)
        output_array = output_data.numpy()
        output_array = output_array.reshape(output_array.shape[-1],output_array.shape[0],output_array.shape[1])
        if i == 0:
            final_output = output_array
        else:
            final_output = np.vstack((final_output,output_array))
#-------------------------------------------------------------------------------------------------------------------------------
if input_conf == 'confd':
    final_output = np.array([])
    final_output_rr = np.array([])
    model_input_shape = (128,3)
    model  = BRUnet_Multi_resp(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confd/best_model_100.h5')
    for i in tqdm.tqdm(range(10)):
        test_loss_list = []
        output_data = tf.convert_to_tensor([])
        output_data_rr = tf.convert_to_tensor([])
        for step , (x_batch_test,y_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
            test_output,test_out_rr = model(x_batch_test)
            test_loss_resp = loss_fn(y_batch_test  , test_output)
            test_loss_rr = loss_fn(x_batch_test_ref_rr , test_out_rr)
            test_loss = test_loss_resp + test_loss_rr
            if step == 0:
                output_data = test_output
                output_data_rr = test_out_rr
            else:
                output_data = tf.concat([output_data , test_output] , axis = 0)
                output_data_rr = tf.concat([output_data_rr , test_out_rr] , axis = 0)
            test_loss_list.append(test_loss)
        output_array = output_data.numpy()
        output_array_rr = output_data_rr.numpy()
        output_array = output_array.reshape(output_array.shape[-1],output_array.shape[0],output_array.shape[1])
        output_array_rr = output_array_rr.reshape(output_array_rr.shape[-1],output_array_rr.shape[0],output_array_rr.shape[1])
        if i == 0:
            final_output = output_array
            final_output_rr = output_array_rr
        else:
            final_output = np.vstack((final_output,output_array))
            final_output_rr = np.vstack((final_output_rr,output_array_rr))
#-------------------------------------------------------------------------------------------------------------------------------
if input_conf == 'confb':
    final_output = np.array([])
    model_input_shape = (128,3)
    model  = BRUnet_Encoder(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confb/best_model_100.h5')
    for i in tqdm.tqdm(range(10)):
        test_loss_list = []
        output_data = tf.convert_to_tensor([])
        for step , (x_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
            output = model(x_batch_test)
            test_loss = loss_fn(x_batch_test_ref_rr , output)
            if step == 0:
                output_data = output
            else:
                output_data = tf.concat([output_data , output] , axis = 0)
            test_loss_list.append(test_loss)
        output_array = output_data.numpy()
        output_array = output_array.reshape(output_array.shape[-1],output_array.shape[0],output_array.shape[1])
        if i == 0:
            final_output = output_array
        else:
            final_output = np.vstack((final_output,output_array))
#-------------------------------------------------------------------------------------------------------------------------------
if input_conf == 'confe':
    final_output = np.array([])
    model_input_shape = (2048,3)
    model  = BRUnet_raw(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confe/best_model_100.h5')
    for i in tqdm.tqdm(range(10)):
        test_loss_list = []
        output_data = tf.convert_to_tensor([])        
        for step , (x_batch_test_raw,y_batch_test) in enumerate(test_dataset):
            output = model(x_batch_test_raw)
            test_loss = loss_fn(y_batch_test , output)
            if step == 0:
                output_data = output
            else:
                output_data = tf.concat([output_data , output] , axis = 0)
            test_loss_list.append(test_loss)
        output_array = output_data.numpy()
        output_array = output_array.reshape(output_array.shape[-1],output_array.shape[0],output_array.shape[1])
        if i == 0:
            final_output = output_array
        else:
            final_output = np.vstack((final_output,output_array))
#-------------------------------------------------------------------------------------------------------------------------------
if input_conf == 'confa':
    final_output = np.array([])
    model_input_shape = (2048,3)
    model  = BRUnet_raw_encoder(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/confa/best_model_100.h5')
    for i in tqdm.tqdm(range(10)):
        test_loss_list = []
        output_data = tf.convert_to_tensor([])
        for step , (x_batch_test_raw, x_batch_test_ref_rr) in enumerate(test_dataset):
            output = model(x_batch_test_raw)
            test_loss = loss_fn(x_batch_test_ref_rr , output)
            if step == 0:
                output_data = output
            else:
                output_data = tf.concat([output_data , output] , axis = 0)
            test_loss_list.append(test_loss)  
        output_array = output_data.numpy()
        output_array = output_array.reshape(output_array.shape[-1],output_array.shape[0],output_array.shape[1])
        if i == 0:
            final_output = output_array
        else:
            final_output = np.vstack((final_output,output_array))
            
#-------------------------------------------------------------------------------------------------------------------------------
if input_conf == 'conff':
    final_output = np.array([])
    final_output_rr = np.array([])
    model_input_shape = (2048,3)
    model  = BRUnet_raw_multi(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/pose1/charan/BR_Uncertainty/MONTE_CARLO/TEST_SAVE_MODEL/conff/best_model_100.h5') 
    for i in tqdm.tqdm(range(10)):
        test_loss_list = []
        output_data = tf.convert_to_tensor([])
        output_data_rr = tf.convert_to_tensor([])
        for step , (x_batch_test_raw , y_batch_test , x_batch_test_ref_rr) in enumerate(test_dataset):
            test_output,test_out_rr = model(x_batch_test_raw)
            test_loss_resp = loss_fn(y_batch_test  , test_output)
            test_loss_rr = loss_fn(x_batch_test_ref_rr , test_out_rr)
            test_loss = test_loss_resp + test_loss_rr
            if step == 0:
                output_data = test_output
                output_data_rr = test_out_rr
            else:
                output_data = tf.concat([output_data , test_output] , axis = 0)
                output_data_rr = tf.concat([output_data_rr , test_out_rr] , axis = 0)
            test_loss_list.append(test_loss)
        output_array = output_data.numpy()
        output_array_rr = output_data_rr.numpy()
        output_array = output_array.reshape(output_array.shape[-1],output_array.shape[0],output_array.shape[1])
        output_array_rr = output_array_rr.reshape(output_array_rr.shape[-1],output_array_rr.shape[0],output_array_rr.shape[1])
        if i == 0:
            final_output = output_array
            final_output_rr = output_array_rr
        else:
            final_output = np.vstack((final_output,output_array))
            final_output_rr = np.vstack((final_output_rr,output_array_rr))
#-------------------------------------------------------------------------------------------------------------------------------            
        
                        
                
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.71s/it]


In [8]:
def extremas_extraction(signal):
    avg_breath_duration = np.array([])
    extrema_relevent = []
    for item in signal:
        amplitude = np.array([])
        pos_peaks , _ = scipy.signal.find_peaks(item , height = [-3000,3000])
        neg_peaks , _ = scipy.signal.find_peaks(-1*item , height = [-3000 , 3000])
        extremas = np.concatenate((pos_peaks , neg_peaks))
        extremas = np.sort(extremas)
        for i in range(len(extremas)):
            amplitude = np.append(amplitude , item[int(extremas[i])])
        amplitude_diff = np.abs(np.diff(amplitude))
        q3 = np.percentile(amplitude_diff , 75)
        threshold = 0.3*q3
        eliminate_pairs_of_extrema = 1
        while(eliminate_pairs_of_extrema):
            amps = np.array([])
            if len(extremas)<3:
                eliminate_pairs_of_extrema = 0
                continue
            for i in range(len(extremas)):
                amps = np.append(amps , item[int(extremas[i])])
            amp_diff = np.abs(np.diff(amps)) 
            min_amp_diff , index = min(amp_diff) , np.argmin(amp_diff)
            #print(min_amp_diff)
            if min_amp_diff > threshold:
                eliminate_pairs_of_extrema = 0
                #extrema_relevent = extremas
            else:
                extremas = np.concatenate((extremas[0:index] , extremas[index+2 :]))
                #amplitude_diff = np.delete(amplitude_diff , index)
        if item[int(extremas[0])] < item[int(extremas[1])]:
            extremas = extremas[1:]
        if item[int(extremas[-1])] < item[int(extremas[-2])]:
            extremas = extremas[:-1]
        no_of_breaths = (len(extremas)-1)/2
        breath_duration = extremas[-1] - extremas[0]
        avg_breath_duration = np.append(avg_breath_duration , breath_duration/no_of_breaths)
        extrema_relevent.append(extremas)
    return avg_breath_duration , extrema_relevent

In [9]:
ref_sig = y_test_data.numpy()
fbpB , fbpA = band_pass(0.1,0.7,8)
final_ref_resp_sig = []
for item in ref_sig:
    final_ref_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
final_ref_resp_sig = np.array(final_ref_resp_sig)

In [10]:
if input_conf == 'confa':
    l = 0.05
    drop_prob = 0.1
    lam = 0.01
    final_output_rr = np.mean(final_output , axis = 0)
    final_var = np.var(final_output , axis = 0)
    tau = (l**2) * (1-drop_prob) / (2. * lam)
    final_var += (tau**-1)
    output_copy = final_output_rr
    std_dev = np.sqrt(final_var)
    #final_output_rr = final_output_rr.reshape(final_output_rr.shape[0],final_output_rr.shape[1])
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    error = np.abs(avg_ref_breath - final_output_rr)
    mae = np.mean(error)
    rmse = np.sqrt(np.mean(error**2))
    print('Mean Absolute Error for {} is: {}'.format(input_conf,mae))
    print('Root Mean Square Error for {} is: {}'.format(input_conf,rmse))
    
    samples = np.arange(0,len(final_output_rr)).reshape(-1,1)
    confa_data = np.hstack((samples,final_output_rr ,final_var ,error))
    col_confa = ['Samples','Final RR Output (BrPM)' ,'Uncertainty','Absolute Error']
    data_confa = pd.DataFrame(confa_data , columns = col_confa)
#----------------------------------------------------------------------------------------------------------------   
if input_conf == 'confb':
    l = 0.005
    drop_prob = 0.1
    lam = 0.005
    final_output_rr = np.mean(final_output , axis = 0)
    final_var = np.var(final_output , axis = 0)
    tau = (l**2) * (1-drop_prob) / (2. * lam)
    final_var += (tau**-1)
    output_copy = final_output_rr
    std_dev = np.sqrt(final_var)
    #final_output_rr = final_output_rr.reshape(final_output_rr.shape[0],final_output_rr.shape[1])
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    error = np.abs(avg_ref_breath - final_output_rr)
    mae = np.mean(error)
    rmse = np.sqrt(np.mean(error**2))
    print('Mean Absolute Error for {} is: {}'.format(input_conf,mae))
    print('Root Mean Square Error for {} is: {}'.format(input_conf,rmse))
    
    samples = np.arange(0,len(final_output_rr)).reshape(-1,1)
    confb_data = np.hstack((samples,final_output_rr , final_var, error))
    col_confb = ['Samples','Final RR Output (BrPM)' , 'Uncertainty', 'Absolute Error']
    data_confb = pd.DataFrame(confb_data , columns = col_confb)
    

In [11]:
if input_conf == 'confc':
    l = 5
    drop_prob = 0.1
    lam = 0.01
    
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    
    final_output_resp = np.mean(final_output , axis = 0)
    final_var = np.var(final_output , axis = 0)
    tau = (l**2) * (1-drop_prob) / (2. * lam)
    final_var += (tau**-1)
    output_copy = final_output_resp
    std_dev = np.sqrt(final_var)
    #final_output_resp = final_output_resp.reshape(final_output_resp.shape[0],final_output_resp.shape[1])
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_breaths = (60*4/duration_resp).reshape(-1,1)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - avg_breaths)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    

    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))
    print(final_var)

#--------------------------------------------------------------------------------------------------------------------    
if input_conf == 'confe':
    l = 1
    drop_prob = 0.1
    lam = 0.005
    
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    final_output_resp = np.mean(final_output , axis = 0)
    final_var = np.var(final_output , axis = 0)
    tau = (l**2) * (1-drop_prob) / (2. * lam)
    final_var += (tau**-1)
    
    output_copy = final_output_resp
    std_dev = np.sqrt(final_var)
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_breaths = (60*4/duration_resp).reshape(-1,1)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - avg_breaths)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))
    
        
    

In [12]:
if input_conf == 'confd':
    l = 5
    drop_prob = 0.1
    lam = 0.01
    
    l_rr = 0.05
    drop_prob_rr = 0.1
    lam_rr = 0.01
    
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    
    final_output_resp = np.mean(final_output,axis = 0)
    final_rr = np.mean(final_output_rr,axis = 0)
    tau = (l**2) * (1-drop_prob) / (2. * lam)
    tau_rr = (l_rr**2) * (1-drop_prob_rr) / (2. * lam_rr)
    
    final_var = np.var(final_output,axis = 0)
    final_var_rr = np.var(final_output_rr,axis = 0)
    final_var += (tau**-1)
    final_var_rr += (tau_rr**-1)
    output_copy = final_output_resp
    output_copy_rr = final_rr
    std_dev = np.sqrt(final_var)
    std_dev_rr = np.sqrt(final_var_rr)
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
        
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - final_rr)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    samples = np.arange(0,len(final_rr)).reshape(-1,1)
    confd_data = np.hstack((samples,final_rr , final_var_rr))
    col_confd = ['Samples','Final RR Output (BrPM)' , 'Uncertainty']
    data_confd = pd.DataFrame(confd_data , columns = col_confd)
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))
    
#-------------------------------------------------------------------------------------------------------
if input_conf == 'conff':
    l = 10
    drop_prob = 0.1
    lam = 0.05
    
    l_rr = 0.05
    drop_prob_rr = 0.1
    lam_rr = 0.05
    
    final_output_resp_sig = []
    inst_br_dur = []
    inst_ref_br_dur = []
    inst_rr = []
    inst_ref_rr = []
    
    final_output_resp = np.mean(final_output,axis = 0)
    final_rr = np.mean(final_output_rr,axis = 0)
    tau = (l**2) * (1-drop_prob) / (2. * lam)
    tau_rr = (l_rr**2) * (1-drop_prob_rr) / (2. * lam_rr)
    final_var = np.var(final_output,axis = 0)
    final_var_rr = np.var(final_output_rr,axis = 0)
    final_var += (tau**-1)
    final_var_rr += (tau_rr**-1)
    output_copy = final_output_resp
    output_copy_rr = final_rr
    std_dev = np.sqrt(final_var)
    std_dev_rr = np.sqrt(final_var_rr)
    for item in final_output_resp:
        final_output_resp_sig.append(scipy.signal.filtfilt(fbpB,fbpA , item))
        
    final_output_resp_sig = np.array(final_output_resp_sig)
    duration_resp,extremas_resp = extremas_extraction(final_output_resp_sig)
    duration_ref_resp,extremas_ref_resp = extremas_extraction(final_ref_resp_sig)
    avg_ref_breath = (60*4/duration_ref_resp).reshape(-1,1)
    
    error_avg_breaths = np.abs(avg_ref_breath - final_rr)
    mae_avg_breath = np.mean(error_avg_breaths)
    rmse_avg_breath = np.sqrt(np.mean(error_avg_breaths**2))
    
    for item in extremas_resp:
        inst_br_dur.append(np.diff(item[0::2]))
    for item in extremas_ref_resp:
        inst_ref_br_dur.append(np.diff(item[0::2]))
    for item in inst_br_dur:
        inst_rr.append(np.mean(60*4/item))
    for item in inst_ref_br_dur:
        inst_ref_rr.append(np.mean(60*4/item))
    
    inst_rr = (np.array(inst_rr)).reshape(-1,1)
    inst_ref_rr = (np.array(inst_ref_rr)).reshape(-1,1)
    
    error_inst_breaths = np.abs(inst_ref_rr - inst_rr)
    mae_inst_breath = np.mean(error_inst_breaths)
    rmse_inst_breath = np.sqrt(np.mean(error_inst_breaths**2))
    
    samples = np.arange(0,len(final_rr)).reshape(-1,1)
    conff_data = np.hstack((samples,final_rr , final_var_rr))
    col_conff = ['Samples','Final RR Output (BrPM)' , 'Uncertainty']
    data_conff = pd.DataFrame(conff_data , columns = col_conff)
    
    print('Mean Absolute Error average wise for {} is: {}'.format(input_conf,mae_avg_breath))
    print('Root Mean Square Error average wise for {} is: {}'.format(input_conf,rmse_avg_breath))
    
    print('Mean Absolute Error instantaneous wise for {} is: {}'.format(input_conf,mae_inst_breath))
    print('Root Mean Square Error instantaneous wise for {} is: {}'.format(input_conf,rmse_inst_breath))



Mean Absolute Error average wise for conff is: 2.3999964148995265
Root Mean Square Error average wise for conff is: 3.0197687783233733
Mean Absolute Error instantaneous wise for conff is: 4.704081278889187
Root Mean Square Error instantaneous wise for conff is: 5.7085605603985865


In [13]:
if input_conf == 'confc' or input_conf == 'confe' or input_conf == 'confd' or input_conf == 'conff':
    no_of_samples = 32*4
    x_unc = np.linspace(start = 0,stop = no_of_samples, num = no_of_samples)
    layout_epistemic = go.Layout(
    title = "Respiratory Waveform with Epistemic Uncertainty for "+ input_conf.upper(),
    yaxis = dict(
        title = 'Output Respiration Signal' 
    ),
    xaxis = dict(
        title = 'samples'
    )
    )
    def update_plot(signals):
        data = []
            # Reference ECG trace
        trace_epistemic = go.Scatter(
            x = x_unc,
            y = final_output_resp_sig[signals], 
            mode = 'lines',
            name = 'Respiration with Epistemic',
                    line = dict(
                    shape = 'spline',
                    color = 'red',
                    width = 5
                     ),
                error_y=dict(
                    type='data', # value of error bar given in data coordinates
                    array=final_var[signals],
                    visible=True,
                    color='black',
                thickness=3,
                width=5)
                )
        fig_epistemic = go.Figure(data = [trace_epistemic],layout = layout_epistemic)
        py.offline.iplot(fig_epistemic)
signals_epsitemic = widgets.IntSlider(min = 0,max = len(final_output_resp_sig), value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_epsitemic)


interactive(children=(IntSlider(value=0, description='Record_no:', max=814), Output()), _dom_classes=('widget-…

In [14]:
if input_conf == 'confb':
    fig1 = px.scatter(data_confb,x = 'Samples', y="Final RR Output (BrPM)",title="Epistemic Uncertainty Distribution for "+input_conf.upper(),
                     color="Uncertainty", color_continuous_scale=px.colors.sequential.Viridis)
    fig1.show()

if input_conf == 'confa':
    fig1 = px.scatter(data_confa,x = 'Samples', y="Final RR Output (BrPM)",title="Epistemic Uncertainty Distribution for "+input_conf.upper(),
                   color="Uncertainty", color_continuous_scale=px.colors.sequential.Viridis)
    fig1.show()

if input_conf == 'confd':
    fig1 = px.scatter(data_confd,x = 'Samples', y="Final RR Output (BrPM)",title="Epistemic Uncertainty Distribution for "+input_conf.upper(),
                     color="Uncertainty", color_continuous_scale=px.colors.sequential.Viridis)

    fig1.show()
     

if input_conf == 'conff':
    fig1 = px.scatter(data_conff,x = 'Samples', y="Final RR Output (BrPM)",title="Epistemic Uncertainty Distribution for "+input_conf.upper(),
                     color="Uncertainty", color_continuous_scale=px.colors.sequential.Viridis)
    fig1.show()

In [15]:
if input_conf == 'confc' or input_conf == 'confe':
    entropy = scipy.stats.entropy(output_copy,base=10)
    print(entropy)
    
if input_conf == 'confb' or input_conf == 'confa':
    entropy = scipy.stats.entropy(output_copy,base=10)
    print(entropy)

if input_conf == 'confd' or input_conf == 'conff':
    entropy = scipy.stats.entropy(output_copy,base=10)
    entropy_rr = scipy.stats.entropy(output_copy_rr,base=10)
    print(entropy)
    print(entropy_rr)

[2.9080546 2.907976  2.9080667 2.9078608 2.907557  2.9080975 2.9079158
 2.90812   2.9074328 2.9076324 2.9080744 2.9081922 2.9078772 2.9081495
 2.9080036 2.9078884 2.9077709 2.908204  2.9077172 2.907881  2.9077952
 2.9078233 2.9077148 2.9078646 2.9080367 2.9078712 2.9080536 2.9079971
 2.908127  2.9077287 2.907987  2.9081438 2.907537  2.9083064 2.9080641
 2.9077883 2.9080374 2.907685  2.907985  2.9078214 2.9077034 2.907869
 2.9073744 2.9080222 2.9080722 2.907897  2.9077444 2.9080179 2.9081514
 2.9082894 2.9079783 2.9082003 2.9079075 2.907746  2.9074621 2.9080975
 2.907859  2.9080853 2.9079874 2.9079437 2.9075978 2.9077804 2.9080675
 2.9081733 2.9079113 2.9081914 2.9080307 2.9080467 2.9078658 2.9079692
 2.9079387 2.9080584 2.9080863 2.9080586 2.907853  2.908178  2.9081724
 2.9081018 2.908371  2.9080606 2.9081655 2.907804  2.9080324 2.9080184
 2.9081328 2.9081216 2.9077845 2.9078639 2.9076734 2.9074996 2.9071968
 2.9080303 2.907887  2.9075458 2.9079056 2.9083006 2.9083233 2.9082735
 2.9079

In [16]:
def Gaussian_NLL(y, mu, sigma, reduce=False):
    #import pdb;pdb.set_trace()
    ax = list(range(1, len(y.shape)))

    logprob = -tf.math.log(sigma) - 0.5*tf.math.log(2*np.pi) - 0.5*((y-mu)/sigma)**2
    loss = tf.reduce_mean(-logprob, axis=ax)
    return tf.reduce_mean(loss) if reduce else loss

In [17]:
if input_conf == 'confc' or input_conf == 'confe':
    nll = Gaussian_NLL(y_test_data, output_copy, std_dev, reduce=False)
    negetive_log = nll.numpy()
    avg_nll = np.mean(negetive_log)
    print("Negetive Log Liklihood for {} is {}".format(input_conf,negetive_log))
    print("Average negetive Log Liklihood for {} is {}".format(input_conf,avg_nll))

if input_conf == 'confb' or input_conf == 'confa':
    nll = Gaussian_NLL(x_test_ref_rr, output_copy, std_dev, reduce=False)
    negetive_log = nll.numpy()
    avg_nll = np.mean(negetive_log)
    print("Negetive Log Liklihood for {} is {}".format(input_conf,negetive_log))
    print("Average negetive Log Liklihood for {} is {}".format(input_conf,avg_nll))

if input_conf == 'confd' or input_conf == 'conff':
    nll = Gaussian_NLL(y_test_data, output_copy, std_dev, reduce=False)
    nll_rr = Gaussian_NLL(x_test_ref_rr, output_copy_rr, std_dev_rr, reduce=False)
    negetive_log = nll.numpy()
    negetive_log_rr = nll_rr.numpy()
    avg_nll = np.mean(negetive_log)
    avg_nll_rr = np.mean(negetive_log_rr)
    print("Negetive Log Liklihood for Respiration Signal for {} is {}".format(input_conf,negetive_log))
    print("Average negetive Log Liklihood  for Respiration Signal for {} is {}".format(input_conf,avg_nll))
    print("Negetive Log Liklihood for Respiration Rate for {} is {}".format(input_conf,negetive_log_rr))
    print("Average negetive Log Liklihood  for Respiration Rate for {} is {}".format(input_conf,avg_nll_rr))

Negetive Log Liklihood for Respiration Signal for conff is [ 0.61837894  2.8192515   0.2987863   0.51293564  0.83118916  0.4283404
  0.9056243   0.8256352   0.82632726  1.1020368   1.3653611   1.5968177
  0.57669604  1.4653394   0.74467826  1.0439922   0.30402917  2.1981764
  1.8298641   0.15996939  0.4156931   0.754079    1.1117139   0.16363059
  0.21159811  2.3680704   1.5983613   5.3721657   8.580663    4.6150675
  3.1825252   2.3123121   1.93842     6.0655746   0.9312668   1.0137538
  0.36341888  1.6199031   9.843379    1.3176281   0.7291973   3.0434713
  2.7442746   1.1025488   3.4261646   3.394413    1.6031837   2.1909225
  3.3949647   3.3376212   2.3705745   5.0564384   4.1700563   4.256546
  2.106236    4.6108      2.690773    2.3493795   2.2124846   4.3942842
  1.1367131   4.5493145   4.520647    3.6125777   2.684256    3.1188176
  2.8177505   7.004591    1.7644224   3.1322863   3.4611442   2.0727792
  1.7554009   0.7433741   2.8142803   0.77045214  4.516413    4.366583
  3.47

In [18]:
std_dev

array([[0.1683118 , 0.16106164, 0.2272147 , ..., 0.12728226, 0.05112832,
        0.03411922],
       [0.20649146, 0.03515371, 0.20822583, ..., 0.07047421, 0.11992398,
        0.13658431],
       [0.20557544, 0.20193163, 0.15814261, ..., 0.1281244 , 0.12970267,
        0.13755938],
       ...,
       [0.03350988, 0.15199444, 0.1576383 , ..., 0.12987614, 0.09289921,
        0.03471817],
       [0.15371953, 0.24395168, 0.03846487, ..., 0.0748873 , 0.15755397,
        0.03333334],
       [0.2045297 , 0.14980611, 0.21057352, ..., 0.13065577, 0.1188025 ,
        0.13658431]], dtype=float32)